[Poprzedni rozdzial](rozdzial_01_wprowadzenie.ipynb)


# 2. Metody symulacyjne

W tej części opisujemy sposób generowania ścieżek procesu GBM oraz konstrukcję różnych estymatorów Monte Carlo użytych w projekcie.

Poniżej przedstawiono krótkie omówienie modeli oraz metod, na których opiera się dalsza analiza.

## 2.1. Model geometrycznego ruchu Browna

Model geometrycznego ruchu Browna (GBM) opisuje ewolucję ceny aktywa jako
$$
S(t) = S(0)\exp(\mu^* t + \sigma B(t)),
$$
gdzie ruch Browna $B(t)$ stanowi źródło losowości. W projekcie ścieżki GBM powstają na podstawie wygenerowanego wektora wartości ruchu Browna o odpowiedniej kowariancji.  
Dla metod redukcji wariancji, takich jak stratyfikacja, stosujemy zmodyfikowane procedury generowania BM.

## 2.2. Wypłata opcji

W projekcie rozważamy dwie wypłaty: europejską oraz dyskretną opcję barierową typu up-and-out. 
Wypłata opcji w ogólnym przypadku dana jest następująco:
$$
A_{n,C} =
\begin{cases}
e^{-r}\,(S(1)-K)^+, & \text{jeśli } S(i/n) < C \ \text{dla wszystkich } i=1,2,\dots,n, \\
0, & \text{w przeciwnym razie}.
\end{cases}
$$
Dla skończonej bariery $C < \infty$ opcja wygasa bez wartości, jeśli ścieżka ceny choć raz przekroczy poziom $C$ w rozważanych punktach czasowych. W przeciwnym przypadku wypłata jest równa zdyskontowanej wartości $(S(1)-K)^+$. 
W przypadku $C=\infty$ warunek barierowy jest zawsze spełniony i powyższa definicja redukuje się do standardowej europejskiej opcji call.

## 2.3. Estymator Crude Monte Carlo

Podstawowy estymator Monte Carlo polega na generowaniu wielu niezależnych ścieżek geometrycznego ruchu Browna, obliczeniu wypłaty dla każdej z nich, a następnie uśrednieniu wyników. Estymator ten ma postać
$$
\hat I_{n,C}^{\mathrm{CMC}}
:=
\frac{1}{R}
\sum_{j=1}^{R}
A_{n,C}^{(j)},
$$
gdzie $A_{n,C}^{(j)}$ oznacza wypłatę obliczoną na podstawie $j$-tej niezależnej realizacji procesu $S(t)$. Metoda ta stanowi punkt odniesienia, względem którego porównywane są bardziej zaawansowane techniki redukcji wariancji.


## 2.4. Estymator stratyfikowany

W tym projekcie estymator stratyfikowany polega na estymacji wartości $I_{n,C}$ poprzez niezależne symulowanie ścieżek geometrycznego ruchu Browna warunkowo na przynależność odpowiadającego im wektora ruchu Browna do zadanych stratum. Estymator ma postać
$$
\hat I_{n,C}^{\mathrm{str}}
=
\sum_{i=1}^{m}
w_i \, \hat I_{n,C}^{(i)},
\qquad
\hat I_{n,C}^{(i)}
=
\frac{1}{R_i}
\sum_{j=1}^{R_i}
A_{n,C}^{(i,j)},
$$
gdzie $A_{n,C}^{(i,j)}$ oznacza wypłatę obliczoną na podstawie $j$-tej realizacji z $i$-tego stratum, $R_i$ jest liczbą symulacji w tym stratum, a $w_i$ są wagami odpowiadającymi poszczególnym stratum. Tak zdefiniowany estymator jest ważoną średnią estymatorów cząstkowych i umożliwia kontrolę rozkładu symulowanych trajektorii oraz redukcję wariancji względem estymatora crude Monte Carlo.


W dalszej części pracy rozważane będą dwie wersje estymatora stratyfikowanego, różniące się sposobem alokacji liczby symulacji pomiędzy poszczególne straty.

### 2.4.1. Rozmiar próby proporcjonalny

W przypadku alokacji proporcjonalnej liczba symulacji w każdej stracie jest proporcjonalna do jej prawdopodobieństwa, tzn.
$$
R_i = R\, w_i,
\qquad i=1,2,\dots,m,
$$
gdzie $w_i = \mathbb{P}(B \in \mathcal S_i)$ oznacza prawdopodobieństwo $i$-tego stratum. Taki wybór alokacji prowadzi do nieobciążonego estymatora i stanowi naturalne uogólnienie estymatora crude Monte Carlo, jednak nie wykorzystuje informacji o zróżnicowaniu wariancji wypłaty pomiędzy poszczególnymi stratum.


### 2.4.2 Rozmiar próby optymalny

Wariancja wynosi
$$
\mathrm{Var}(\hat I_{n,C}^{\mathrm{str}})
=
\sum_{i=1}^m \frac{w_i^2\,\sigma_i^2}{R_i},
\qquad
\sigma_i^2=\mathrm{Var}(A_{n,C}\mid B\in \mathcal S_i).
$$

Minimalizację wariancji przy zadanym budżecie
$$
R=\sum_{i=1}^m R_i
$$
zapewnia alokacja Neymana
$$
R_i^*
=
R\cdot
\frac{w_i\,\sigma_i}{\sum_{j=1}^m w_j\,\sigma_j},
$$
która w rozważanym przypadku ($w_i=1/m$) upraszcza się do
$$
R_i^*
=
R\cdot
\frac{\sigma_i}{\sum_{j=1}^m \sigma_j}.
$$

W praktyce wariancje $\sigma_i^2$ są nieznane, dlatego stosujemy procedurę
dwustopniową: wstępny *pilot run* w każdej warstwie w celu oszacowania
$\hat\sigma_i$, a następnie właściwą symulację z alokacją $\hat R_i$
otrzymaną przez podstawienie $\hat\sigma_i$ do wzoru Neymana. Tak
skonstruowany estymator minimalizuje wariancję w klasie estymatorów
stratyfikowanych przy zadanym budżecie symulacji.

## 2.5. Estymator antytetyczny

Metoda antytetyczna polega na generowaniu par $(Z,-Z)$, co odpowiada konstrukcji
$Z=\Phi^{-1}(U)$ oraz $-Z=\Phi^{-1}(1-U)$ dla $U\sim U(0,1)$. Estymator ma postać
$$
\hat I_{1, \infty}^{\mathrm{anti}}
=\frac{1}{R}\sum_{i=1}^{R/2}\big(f(Z_i)+f(-Z_i)\big),
$$
gdzie $f(Z)=e^{-r}\big(S(0)e^{\mu^*+\sigma Z}-K\big)_+=A_{1,\infty}$.  Standardowy estymator Monte Carlo ma wariancję
$$
\mathrm{Var}(\hat I_{1, \infty}^{\mathrm{CMC}})=\frac{\mathrm{Var}(f(Z))}{R}.
$$

Wariancja estymatora antytetycznego spełnia zależność
$$
\mathrm{Var}(\hat I_{1, \infty}^{\mathrm{anti}})
=
\frac{\mathrm{Var}(f(Z))}{R}
\big(1+\mathrm{Corr}(f(Z),f(-Z))\big).
$$

Funkcja $f(Z)$ jest niemalejąca względem $Z$, ponieważ jest złożeniem funkcji
wykładniczej i funkcji $(\cdot)_+$. Zgodnie z lematami ze slajdów, dla funkcji
monotonicznych zachodzi
$$
\mathrm{Corr}(f(Z),f(-Z))\le 0.
$$

W konsekwencji
$$
\mathrm{Var}(\hat I_{1, \infty}^{\mathrm{anti}})
\le
\mathrm{Var}(\hat I_{1, \infty}^{\mathrm{CMC}}),
$$
co uzasadnia redukcję wariancji przy zastosowaniu par antytetycznych.

## 2.6. Estymator zmiennej kontrolnej

W metodzie *control variate* wykorzystujemy dodatkową zmienną losową $Z$, dla której znana jest wartość oczekiwana oraz która jest silnie skorelowana z estymowaną zmienną $A_{1,\infty}$.  
W niniejszym projekcie jako zmienną kontrolną przyjmujemy
$$
Z = B(1),
$$
czyli wartość ruchu Browna w chwili $t=1$. Z własności ruchu Browna wynika, że $B(1)\sim \mathcal N(0,1)$, $\mathbb{E}[Z]=0.$ Wybór ten jest naturalny, ponieważ zarówno cena $S(1)$, jak i payoff opcji są funkcjami $B(1)$, co implikuje istotną korelację pomiędzy $Z$ i $A_{1,\infty}$.

#### Konstrukcja estymatora *control variate*

Dla $R$ niezależnych symulacji generujemy próbki
$$
Z_j = B(1)_j \sim \mathcal N(0,1), \qquad
A_{1, \infty}^{(j)} = e^{-r}\big(S(0)e^{\mu^* + \sigma Z_j} - K\big)_+,
\quad j=1,\dots,R.
$$

Estymator z zastosowaniem zmiennej kontrolnej definiujemy jako
$$
\hat I^{\text{CV}}_{1,\infty}
=
\hat I^{\text{CMC}}_{1,\infty}
+
\hat c \left(\frac{1}{R}\sum_{j=1}^R Z_j - \mathbb{E}[Z]\right).
$$
Ponieważ $\mathbb{E}[Z]=0$, wzór upraszcza się do
$$
\hat I^{\text{CV}}_{1,\infty}
=
\frac{1}{R}\sum_{j=1}^R A_{1, \infty}^{(j)}
+
\hat c\,\frac{1}{R}\sum_{j=1}^R Z_j.
$$

Współczynnik $\hat c$ estymujemy z próby zgodnie z klasyczną formułą
$$
\hat c
=
-
\frac{\widehat{\mathrm{Cov}}(A_{1,\infty},Z)}{\widehat{\mathrm{Var}}(Z)},
$$
gdzie
$$
\widehat{\mathrm{Cov}}(A_{1,\infty},Z)
=
\frac{1}{R-1}\sum_{j=1}^R \big(A_{1, \infty}^{(j)}-\bar A_{1,\infty}\big)(Z_j-\bar Z),
\qquad
\widehat{\mathrm{Var}}(Z)
=
\frac{1}{R-1}\sum_{j=1}^R (Z_j-\bar Z)^2.
$$


[Nastepny rozdzial](rozdzial_03_black_scholes.ipynb)
